In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# CUDA 사용 가능한지 확인
import torch
print("CUDA enabled:", torch.cuda.is_available())

CUDA enabled: True


In [3]:
import os

# 경로 설정
folder_path = '/content/drive/MyDrive/IOT/Human/news/wav'
os.makedirs(folder_path, exist_ok=True)
folder_path = '/content/drive/MyDrive/IOT/Human/news/txt'
os.makedirs(folder_path, exist_ok=True)

folder_path = '/content/drive/MyDrive/IOT/Human/drama/wav'
os.makedirs(folder_path, exist_ok=True)
folder_path = '/content/drive/MyDrive/IOT/Human/drama/txt'
os.makedirs(folder_path, exist_ok=True)

folder_path = '/content/drive/MyDrive/IOT/Human/comics/wav'
os.makedirs(folder_path, exist_ok=True)
folder_path = '/content/drive/MyDrive/IOT/Human/comics/txt'
os.makedirs(folder_path, exist_ok=True)

folder_path = '/content/drive/MyDrive/IOT/AI/STS2/wav'
os.makedirs(folder_path, exist_ok=True)
folder_path = '/content/drive/MyDrive/IOT/AI/STS2/txt'
os.makedirs(folder_path, exist_ok=True)

In [4]:
# @title
# 처음엔 이거 코드 돌리셔야 해요!

!pip install pytube
!pip install torch torchvision torchaudio
!pip install --upgrade transformers accelerate
!pip install --upgrade transformers accelerate scipy
!pip install korean-romanizer
# https://github.com/SYSTRAN/faster-whisper
# openai-whisper 설치
!pip install -U openai-whisper
!apt-get install ffmpeg
!pip install ffmpeg-python
!pip install git+https://github.com/openai/whisper.git
!apt-get install ffmpeg
!pip install ffmpeg-python
# VITS 설치
!apt install git-lfs
!git lfs install
!git clone https://huggingface.co/spaces/skytnt/moe-tts
!pip install gradio==3.40.1
!pip install -r requirements.txt
!pip install transformers torch scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

공유 드라이브

https://drive.google.com/drive/folders/140pPZdv8JmRjjxACucVLDzR4roaJ-p6P?usp=sharing

# Whisper (STT)

## 모델 input (인간음성)

- wav_paths : 인간음성파일들 경로 모음

In [ ]:
# 사람 음성 파일 담긴 폴더 경로
directory = '/content/drive/MyDrive/IOT/Human/news/wav'

In [ ]:
def get_file_paths(directory):
    file_paths = []

    # directory 내의 모든 파일 및 디렉터리 목록을 가져옵니다.
    for file_name in os.listdir(directory):
        # 파일의 전체 경로를 생성합니다.
        full_path = os.path.join(directory, file_name)

        # 경로가 파일인지 확인하고 리스트에 추가합니다.
        if os.path.isfile(full_path):
            file_paths.append(full_path)

    return file_paths

wav_paths = get_file_paths(directory)

## Whisper 실행하기

숨겨진 코드는 Whisper (STT) 불러오는 코드

In [ ]:
# @title
# https://github.com/SYSTRAN/faster-whisper
# openai-whisper 설치
!pip install -U openai-whisper
!apt-get install ffmpeg
!pip install ffmpeg-python
!pip install git+https://github.com/openai/whisper.git
!apt-get install ffmpeg
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=77e18934656e95697bddc5be04c08e0f53d4ca6c2f1e77e1373421ffcfe14a04
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-nb9o9o72
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper

In [ ]:
import os
import whisper
import re

# Whisper 모델 로드
model = whisper.load_model("medium")

CUDA enabled: True


100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 110MiB/s]


In [ ]:
# Speech -> Text
txt_paths=[]

for wav_path in wav_paths:
  result = model.transcribe(wav_path, language='Korean')
  output_text_file_path = re.sub(r'/wav/', '/txt/', wav_path)
  output_text_file_path = output_text_file_path.rsplit('.', 1)[0] +'.txt'

  with open(output_text_file_path, 'w', encoding='utf-8') as f:
    f.write(result['text'])

  print(f"Transcription completed. The result is saved in {output_text_file_path}")

Transcription completed. The result is saved in /content/drive/MyDrive/IOT/Human/news/txt/공의 실밥까지 확인하고 송구하는 베테랑 선수의 여유 shorts 최강야구.txt
Transcription completed. The result is saved in /content/drive/MyDrive/IOT/Human/news/txt/1997일 만에 등판해서 최고 구속 경신한 43세 니퍼트 shorts 최강야구.txt
Transcription completed. The result is saved in /content/drive/MyDrive/IOT/Human/news/txt/감독이 고의 사구 두 번으로 무사 만루로 만든 이유 shorts 최강야구.txt
Transcription completed. The result is saved in /content/drive/MyDrive/IOT/Human/news/txt/니퍼트도 극찬한 유희관의 지리는 투구 shorts 최강야구.txt
Transcription completed. The result is saved in /content/drive/MyDrive/IOT/Human/news/txt/기술적으로 들어올려서 안타치는 이대호 shorts 최강야구.txt
Transcription completed. The result is saved in /content/drive/MyDrive/IOT/Human/news/txt/거기서 왜 2루로 던져 shrots 최강야구.txt
Transcription completed. The result is saved in /cont

speech를 text로 바꾼 후 경로는 ...

- output_text_file_path

# Grad-TTS

숨긴 코드는 VITS 사용하기 위한 코드

In [ ]:
# @title
!pip install gtts
!pip install playsound
!pip install pygobject
!pip install pydub
!pip install simpleaudio
!pip install pygame

input (text 파일) : '/content/drive/MyDrive/IOT/Human/news/txt'

output (wav 파일) : '/content/drive/MyDrive/IOT/AI/STS2/wav'

모델에 입력: file_paths (경로들모음)

In [ ]:
from gtts import gTTS
import os

def text_files_to_speech(input_folder, output_folder, lang='ko', slow=False):
    # 입력 폴더에서 텍스트 파일들을 불러옴
    text_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f)) and f.endswith('.txt')]

    # 출력 폴더가 존재하지 않으면 생성
    os.makedirs(output_folder, exist_ok=True)

    for text_file in text_files:
        # 텍스트 파일의 전체 경로
        input_path = os.path.join(input_folder, text_file)

        # 텍스트 파일 읽기
        with open(input_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # gTTS 객체 생성
        tts = gTTS(text=text, lang=lang)

        # 출력 파일 경로
        output_file = os.path.splitext(text_file)[0] + '.wav'
        output_path = os.path.join(output_folder, output_file)

        # 음성 파일로 저장
        tts.save(output_path)
        print(f'Saved {output_path}')

In [ ]:
# 경로 설정
input_folder = '/content/drive/MyDrive/IOT/Human/news/txt'
output_folder = '/content/drive/MyDrive/IOT/AI/STS2/wav'

# 함수 호출
text_files_to_speech(input_folder, output_folder)

Saved /content/drive/MyDrive/IOT/AI/STS2/wav/김성근 감독님만 등장하면 느와르로 변하는 최강야구ㅋㅋㅋ shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/니퍼트도 극찬한 유희관의 지리는 투구 shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/공의 실밥까지 확인하고 송구하는 베테랑 선수의 여유 shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/기술적으로 들어올려서 안타치는 이대호 shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/감독님이 불편해하는() 선수가 문자 보내면 생기는 일ㅋㅋㅋ shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/감독이 고의 사구 두 번으로 무사 만루로 만든 이유 shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/공 3개로 타자 멘탈 나가게 만드는 유희관 shorts 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/거기서 왜 2루로 던져 shrots 최강야구.wav
Saved /content/drive/MyDrive/IOT/AI/STS2/wav/최강야구 들어오자마자 슈

In [ ]:
from IPython.display import Audio
Audio('/content/drive/MyDrive/IOT/AI/STS2/wav/1997일 만에 등판해서 최고 구속 경신한 43세 니퍼트 shorts 최강야구.wav', autoplay=True)